In [ ]:
# 🚀 PRODUCTION SERVING ENDPOINT - NEW WORKFLOW (CONFIG-DRIVEN)

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput,
    ServedEntityInput
)
import mlflow
from mlflow.tracking import MlflowClient
import time
import sys
import yaml
import traceback
import requests
from typing import Optional, Dict, Tuple
from datetime import datetime

print("=" * 80)
print("🚀 PRODUCTION SERVING ENDPOINT (NEW WORKFLOW)")
print("=" * 80)

# ✅ LOAD PIPELINE CONFIGURATION

print("\n📋 Step 1: Loading configuration from pipeline_config.yml...")

try:
    import os

    config_path = "/Workspace/Repos/vipultak7171@gmail.com/ml-credit-risk/dev_env/pipeline_config.yml"

    if not os.path.exists(config_path):
        config_path = "/Workspace/ml-credit-risk/dev_env/pipeline_config.yml"

    with open(config_path, "r") as f:
        pipeline_cfg = yaml.safe_load(f)

    print(f"✅ Configuration loaded successfully from: {config_path}")

except FileNotFoundError:
    print("❌ ERROR: pipeline_config.yml not found!")
    print("💡 Please ensure pipeline_config.yml is in the notebook directory")
    dbutils.notebook.exit("FAILURE")
except Exception as e:
    print(f"❌ ERROR loading configuration: {e}")
    traceback.print_exc()
    dbutils.notebook.exit("FAILURE")

# ================== CONFIGURATION CLASS ==================

class Config:
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]
        
        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.MODEL_TYPE = MODEL_TYPE
        
        self.ENDPOINT_NAME = f"{BASE_NAME.replace('_', '-')}-{MODEL_TYPE}-prod"
        
        self.PRODUCTION_ALIAS = pipeline_cfg["aliases"]["production"]
        self.STAGING_ALIAS = pipeline_cfg["aliases"]["staging"]
        
        serving_cfg = pipeline_cfg.get("serving", {})
        self.WORKLOAD_SIZE = serving_cfg.get("workload_size", "Small")
        self.SCALE_TO_ZERO = serving_cfg.get("scale_to_zero_enabled", True)
        self.MIN_PROVISIONED_THROUGHPUT = serving_cfg.get("min_provisioned_throughput", 0)
        self.MAX_PROVISIONED_THROUGHPUT = serving_cfg.get("max_provisioned_throughput", 0)
        
        self.WORKLOAD_TYPE = serving_cfg.get("workload_type", "CPU")
        self.ENVIRONMENT_VARS = serving_cfg.get("environment_vars", {})
        
        self.DEPLOYMENT_TIMEOUT = serving_cfg.get("deployment_timeout", 1800)
        self.STATUS_CHECK_INTERVAL = serving_cfg.get("status_check_interval", 15)
        
        self.SLACK_ENABLED = pipeline_cfg["notifications"]["enabled"]
        self.SLACK_WEBHOOK_URL = self._get_slack_webhook()

    def _get_slack_webhook(self):
        if not self.SLACK_ENABLED:
            return None
        try:
            scopes = ["shared-scope", "dev-scope", "prod-scope", "ml-scope"]
            for s in scopes:
                try:
                    return dbutils.secrets.get(s, "SLACK_WEBHOOK_URL")
                except:
                    pass
            return None
        except:
            return None

config = Config()

# ================== SLACK HANDLER ==================

class SlackNotifier:
    def __init__(self, url):
        self.url = url
        self.enabled = bool(url)

    def send(self, message, level="info", extra=None):
        if not self.enabled:
            print(f"[SLACK OFF] {message}")
            return
        emoji = {"info": "ℹ️","success": "✅","warning": "⚠️","error": "❌","rocket": "🚀","gear": "⚙️"}.get(level,"ℹ️")
        text = f"{emoji} *{message}*"
        if extra:
            text += "\n" + "\n".join([f"• *{k}:* {v}" for k,v in extra.items()])
        try:
            requests.post(self.url, json={"text": text}, timeout=5)
        except:
            pass

slack = SlackNotifier(config.SLACK_WEBHOOK_URL)

# ================== CLIENT INIT ==================

print("\n🔧 Initializing Clients...")
try:
    w = WorkspaceClient()
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
except Exception as e:
    slack.send("Client Init Failed", "error", {"Reason": str(e)})
    dbutils.notebook.exit("FAILURE")

# ================== MODEL FETCH ==================

def get_production_version():
    try:
        mv = client.get_model_version_by_alias(config.MODEL_NAME, config.PRODUCTION_ALIAS)
        return int(mv.version)
    except:
        return None

# ================== ENDPOINT HANDLING ==================

def check_endpoint_exists(name):
    try:
        w.serving_endpoints.get(name=name)
        return True
    except:
        return False

def deploy_endpoint(version, exists):
    served = ServedEntityInput(
        entity_name=config.MODEL_NAME,
        entity_version=str(version),
        workload_size=config.WORKLOAD_SIZE,
        scale_to_zero_enabled=config.SCALE_TO_ZERO
    )
    try:
        if exists:
            w.serving_endpoints.update_config(
                name=config.ENDPOINT_NAME,
                served_entities=[served]
            )
        else:
            w.serving_endpoints.create(
                name=config.ENDPOINT_NAME,
                config=EndpointCoreConfigInput(served_entities=[served])
            )
        slack.send("Deployment Started", "gear", {"Endpoint": config.ENDPOINT_NAME, "Version": version})
        return True
    except Exception as e:
        slack.send("Deployment Failed", "error", {"Reason": str(e)})
        return False

def wait_ready(version):
    start = time.time()
    while time.time() - start < config.DEPLOYMENT_TIMEOUT:
        ep = w.serving_endpoints.get(name=config.ENDPOINT_NAME)
        if "READY" in str(ep.state.ready):
            duration = round(time.time() - start,2)
            slack.send("Serving Endpoint Ready", "rocket", {"Version": version, "Time": f"{duration}s"})
            return True
        time.sleep(config.STATUS_CHECK_INTERVAL)
    slack.send("Timeout Waiting For Endpoint", "error")
    return False

# ================== MAIN ==================

def main():
    version = get_production_version()
    if not version:
        slack.send("No Production Model Found", "error")
        return "FAILURE"

    exists = check_endpoint_exists(config.ENDPOINT_NAME)

    if not deploy_endpoint(version, exists):
        return "FAILURE"

    if not wait_ready(version):
        return "FAILURE"

    slack.send("Deployment Completed", "success", {"Endpoint": config.ENDPOINT_NAME, "Version": version})
    return "SUCCESS"

# ================== RUN ==================

result = main()
try:
    dbutils.notebook.exit(result)
except:
    print(result)
